
# COS30049 - Assignment 2
### Session 26 Group 2
### Swinburne Univeristy of Technology

# 1.0 Data Collection

## 1.1 Verify the DataSet Can Be Read
Checks to verify:
- File exists
- Columns pesent
- No missing values

In [22]:
import pandas as pd
import os

In [15]:
file_path = "Constraint_English_Train.xlsx"

if os.path.exists(file_path):
    misinfo_data = pd.read_excel(file_path)
    print("Dataset loaded successfully!")
    print("Columns:", misinfo_data.columns.tolist())
    print(misinfo_data.head())
else:
    print("File not found:", file_path)

Dataset loaded successfully!
Columns: ['id', 'tweet', 'label']
   id                                              tweet label
0   1  The CDC currently reports 99031 deaths. In gen...  real
1   2  States reported 1121 deaths a small rise from ...  real
2   3  Politically Correct Woman (Almost) Uses Pandem...  fake
3   4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4   5  Populous states can generate large case counts...  real


## 1.2 Test Retrieval of Data Matching "CDC"

In [ ]:
import pandas as pd
import re

file_path = "Constraint_English_Train.xlsx"
misinfo_data = pd.read_excel(file_path)

# Compile the regex pattern
pattern = re.compile(r'CDC', re.IGNORECASE)  # ignore case if needed

# Example: check in a specific column, e.g., 'text'
if 'text' in misinfo_data.columns:
    matches = misinfo_data['text'].apply(lambda x: bool(pattern.search(str(x))))
    print("Rows containing 'CDC':")
    print(misinfo_data[matches])
else:
    # If you want to search all columns
    matches = misinfo_data.applymap(lambda x: bool(pattern.search(str(x))))
    print("Rows containing 'CDC':")
    print(misinfo_data[matches.any(axis=1)])

Rows containing 'CDC':
        id                                              tweet label
0        1  The CDC currently reports 99031 deaths. In gen...  real
6        7  If you tested positive for #COVID19 and have n...  real
27      28  Just Appendix B gathering all the state orders...  real
33      34  CDC Recommends Mothers Stop Breastfeeding To B...  fake
138    139  Youth sports organizations: As you resume acti...  real
...    ...                                                ...   ...
6338  6339  ???The CDC can detain anyone with a fever ??" ...  fake
6345  6346  1645 deaths were reported today bringing the t...  real
6377  6378  Acc to @CDCgov &amp; @WHO there is currently n...  real
6391  6392  The CDC ???adjusted the US Covid deaths from 1...  fake
6405  6406  The cloth face coverings recommended to slow s...  real

[281 rows x 3 columns]


/var/folders/w0/r8xr9pyn4w19x3x9z5x82hm00000gn/T/ipykernel_96455/49314103.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  matches = misinfo_data.applymap(lambda x: bool(pattern.search(str(x))))


## 1.3 Combine data sets
- Datasets have been cleaned in Google Refine

In [20]:
train_df = pd.read_excel("Constraint_English_Train_GR.xlsx")
val_df = pd.read_excel("Constraint_English_Val_GR.xlsx")
test_df = pd.read_excel("Constraint_English_Test_GR.xlsx")

/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 2.0 Data preprocessing

## 2.1 Emoji and Symbols Refining

### Test Emoji and Symbols Detection
- Keep symbols used in natural language

In [ ]:
import pandas as pd
import re

df = pd.DataFrame({
    'text': ["Hello world", "Hi there 😀", "@user_name is cool", "No emojis here!", "symbol �"]
})

pattern = re.compile(r'[^\w\s@.,!?#]', flags=re.UNICODE)
df['has_emoji'] = df['text'].apply(lambda x: bool(pattern.search(str(x))))

print(df)

                 text  has_emoji
0         Hello world      False
1          Hi there 😀       True
2  @user_name is cool      False
3     No emojis here!      False
4            symbol �       True


### Test emoji refining

In [24]:
import pandas as pd
import re

df = pd.DataFrame(train_df)

# Step 1: Clean text safely
def clean_text(text):
    text = str(text)
    
    # Step 1a: Protect @usernames and #hashtags
    placeholders = {}
    for match in re.findall(r'(@\w+|#\w+)', text):
        ph = f"PLACEHOLDER{len(placeholders)}"
        placeholders[ph] = match
        text = text.replace(match, ph)
    
    # Step 1b: Replace all underscores with spaces
    text = text.replace('_', ' ')
    
    # Step 1c: Remove emojis / unusual symbols
    # Keep letters, numbers, whitespace, @, #, ., ,, !, ?
    text = re.sub(r'[^\w\s@.,!?#]', '', text, flags=re.UNICODE)
    
    # Step 1d: Remove leading punctuation (like . , ! ?) at start of text
    text = re.sub(r'^[.,!?_\s]+', '', text)
    
    # Step 1e: Restore usernames and hashtags
    for ph, original in placeholders.items():
        text = text.replace(ph, original)
    
    return text

# Apply cleaning function to the 'tweet' column
df['clean_text'] = df['tweet'].apply(clean_text)

# Optional: preview result
print(df[['tweet', 'clean_text']])

# Save cleaned file
df.to_excel("Constraint_English_Train_Cleaned.xlsx", index=False)

                                                  tweet  \
0     The CDC currently reports 99031 deaths. In gen...   
1     States reported 1121 deaths a small rise from ...   
2     Politically Correct Woman (Almost) Uses Pandem...   
3     #IndiaFightsCorona: We have 1524 #COVID testin...   
4     Populous states can generate large case counts...   
...                                                 ...   
6415  A tiger tested positive for COVID-19 please st...   
6416  Autopsies prove that COVID-19 is a blood clot,...   
6417  _A post claims a COVID-19 vaccine has already ...   
6418  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund   
6419  It has been 93 days since the last case of COV...   

                                             clean_text  
0     The CDC currently reports 99031 deaths. In gen...  
1     States reported 1121 deaths a small rise from ...  
2     Politically Correct Woman Almost Uses Pandemic...  
3     #IndiaFightsCorona We have 1524 #COVID testing...  
4

### Clean emojis on all 3 datasets

- Normalized usernames and hashtags instead of keeping them.

In [33]:
import pandas as pd
import re

# Step 1: Define cleaning function
def clean_text(text):
    text = str(text)
    
    # Step 1a: Normalize links, usernames and hashtags 
    # Normalize @usernames to <USER>
    text = re.sub(r'@\w+', '<USER>', text)

    # Normalize #hashtags to <HASHTAG:topic>
    def replace_hashtag(match):
        hashtag = match.group()[1:]  # remove #
        return f"<HASHTAG:{hashtag}>"
    
    text = re.sub(r'#\w+', replace_hashtag, text)
    
    # Normalize links
    text = re.sub(r'https?://\S+|www\.\S+', '<LINK>', text)
    
    # Step 1b: Replace all underscores 
    text = text.replace('_', '')
    
    # Step 1c: Remove emojis / unusual symbols
    # Keep letters, numbers, whitespace, @, #, ., ,, !, ?
    text = re.sub(r'[^\w\s@.,!?#<>:;]', '', text, flags=re.UNICODE)
    
    # Step 1d: Remove leading punctuation (like . , ! ?) at start of text
    text = re.sub(r'^[.,!?_\s]+', '', text)
    
    return text


# Step 2: Define helper function for cleaning datasets
def clean_dataset(input_path, output_path, text_column='tweet'):
    """
    Reads a dataset, cleans the specified text column, and saves cleaned dataset.
    """
    df = pd.read_excel(input_path)

    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in dataset.")

    df['clean_text'] = df[text_column].apply(clean_text)

    df.to_excel(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")


# Step 3: Clean train, validation, and test datasets
datasets = [
    ("Constraint_English_Train.xlsx", "Constraint_English_Train_Cleaned.xlsx"),
    ("Constraint_English_Val_GR.xlsx", "Constraint_English_Val_Cleaned.xlsx"),
    ("Constraint_English_Test_GR.xlsx", "Constraint_English_Test_Cleaned.xlsx")
]

for input_path, output_path in datasets:
    clean_dataset(input_path, output_path)

Cleaned dataset saved to Constraint_English_Train_Cleaned.xlsx
Cleaned dataset saved to Constraint_English_Val_Cleaned.xlsx
Cleaned dataset saved to Constraint_English_Test_Cleaned.xlsx


/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 2.2 Remove Stopwords
- use nltk library

In [34]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /Users/zara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[nltk_data] Downloading package stopwords to /Users/zara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cleaned dataset saved to Constraint_English_Train_Cleaned.xlsx
Cleaned dataset saved to Constraint_English_Val_Cleaned.xlsx
Cleaned dataset saved to Constraint_English_Test_Cleaned.xlsx


/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## 1.6 Lemmatization
- Lemmatization cleans word forms that has "-ing" so that words are in keyword forms.
- uses Spacy or NLTK
- choice: Spacy (more accurate and faster, but larger)

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-"])

## 2.3 Spelling Correction
- Tools available: TextBlob, SymSpell, Hunspell or pyspellchecker
- Choice: TextBlob is slow for large datasets, but since our datasets are small it is suitable for the ML.

In [ ]:
from textblob import TextBlob

def correct_spelling(text):
    return str(TextBlob(text).correct())

In [ ]:
import nltk
nltk.download('punkt', download_dir="/Users/zara/nltk_data")
nltk.data.path.append("/Users/zara/miniconda3/envs/ctip")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zara/miniconda3/envs/ctip/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

- conda info --envs
- replace /Users..... to your own path

## 2.4 Improved final code
- added hashtag, link and user mention normalization, word lemetization (+ removed spaces due to lemetization), and spell checking.

Tools:
1. pandas - reading and writing excel datasets (pd.red_excel, df.to_excel)
2. re (regex) - Text normalization (usernames, hashtags, links) and symbol removal.
3. nltk - stopword removal 
4. spacy - lemmatization 
5. textblob - spelling correction
6. lambda + regex - placeholder integrity


Preprocessing steps taken:
- username to <USER>
- hashatags to <HASHTAG:topic>
- links to <LINK>
- removed underscores
- removed emojis/weird symbols 
- removed leading punctuations
- removed stopwords - split text intp tokens, filter stopwords and rejoin
- lemmatization - convert words to their root form (avoid pronouns)
- remove spaces in placeholders after lemmetization
- spelling correction 

Dataset Processing
- Reads each dataset (pd.read_excel)
- Applies cleaning to a specified column (tweet)
- Writes cleaned data to a new Excel file.
- Processes multiple datasets in a loop.


Observations: there are still some spaces in placeholders <> after lemmetization even when spaces are removed with regex.

In [80]:
import pandas as pd
import re
import nltk
import spacy
from textblob import TextBlob

# Lemmatize
nlp = spacy.load("en_core_web_sm")

def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if token.lemma_ != "-PRON-"])

# Remove spaces from lemmaitization
def remove_spaces_in_placeholders(text):
    return re.sub(r'<\s*(.*?)\s*>', lambda m: f"<{m.group(1).replace(' ', '')}>", text)

# Spelling correction
def correct_spelling(text):
    return str(TextBlob(text).correct())

# Hashtag normalization function
def replace_hashtag(match):
    hashtag = match.group()[1:].lower()
    return f"<HASHTAG:{hashtag}>"

# Download stopwords once
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
negations = {"no", "not", "nor", "never"}
stop_words -= negations

# Cleaning function
def clean_text(text, do_lemmatize=True, do_spellcheck=False):
    text = str(text)

    # Normalize usernames, hashtags, and links
    text = re.sub(r'@\w+', '<USER>', text)
    text = re.sub(r'#\w+', replace_hashtag, text)
    text = re.sub(r'https?://\S+|www\.\S+', '<LINK>', text)

    # Remove underscores
    text = text.replace('_', '')

    # Remove emojis/unusual symbols but keep punctuation
    text = re.sub(r'[^\w\s@.,!?#<>:;]', '', text, flags=re.UNICODE)

    # Remove leading punctuation
    text = re.sub(r'^[.,!?_\s]+', '', text)

    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word.lower() not in stop_words]
    text = " ".join(tokens)

    # Lemmatize and remove spaces 
    if do_lemmatize:
        text = lemmatize_text(text)
    text = remove_spaces_in_placeholders(text)

    # Spellcheck
    if do_spellcheck:
        text = correct_spelling(text)

    return text

# Dataset cleaning helper
def clean_dataset(input_path, output_path, text_column='tweet', do_lemmatize=True, do_spellcheck=False):
    df = pd.read_excel(input_path)

    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' not found in dataset.")

    df['clean_text'] = df[text_column].apply(lambda x: clean_text(x, do_lemmatize, do_spellcheck))

    df.to_excel(output_path, index=False)
    print(f"Cleaned dataset saved to {output_path}")

# Run cleaning for datasets
datasets = [
    ("Constraint_English_Train_GR.xlsx", "Constraint_English_Train_Cleaned.xlsx"),
    ("Constraint_English_Val_GR.xlsx", "Constraint_English_Val_Cleaned.xlsx"),
    ("Constraint_English_Test_GR.xlsx", "Constraint_English_Test_Cleaned.xlsx")
]

for input_path, output_path in datasets:
    clean_dataset(input_path, output_path)

[nltk_data] Downloading package stopwords to /Users/zara/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Cleaned dataset saved to Constraint_English_Train_Cleaned.xlsx


/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Cleaned dataset saved to Constraint_English_Val_Cleaned.xlsx


/opt/anaconda3/envs/ctip/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Cleaned dataset saved to Constraint_English_Test_Cleaned.xlsx
